## This is an example showcasing the loading of trARPES data as collected using the Phoibos detector at FHI Berlin.
The band dispersion is loaded as a xarray dataframe following a conversion to the [NeXus format](https://manual.nexusformat.org/classes/contributed_definitions/NXmpes.html#nxmpes) using the [Nomad Parser Nexus](https://github.com/nomad-coe/nomad-parser-nexus).

First, the SpecsScan class is imported which has the scan loader as its class method.

In [ ]:
%load_ext autoreload
%autoreload 2
from specsscan import SpecsScan
import matplotlib.pyplot as plt

%matplotlib widget

Here, a SpecsScan class instance is created as per the configuration provided in [config.yaml](../tests/data/config.yaml). The user may set the entries in config.yaml file, for example, the data path and conversion parameters as per the requirements before creating this instance

### The path may changed to point to the scan folder of the data of interest (for example, on a server drive)

In [ ]:
sps = SpecsScan(config="../tests/data/config.yaml")
path = "../tests/data/" # Path to the test data set

The load_scan method loads the scan as an xarray along with the metadata needed for nexus conversion. The progress bars can be activated by changing the config parameter, enable_nested_progress_bar, to true in config.yaml. Additionally, the data can be cropped by passing a boolean "crop" to the loader, provided the crop parameters already exist in the given instance. 

In [ ]:
res_xarray = sps.load_scan(
    scan=4450, # Scan number for an example mirror scan
    path = path,
    crop=True
)

In [ ]:
plt.figure()
res_xarray[:,:,0].plot()

The loader has given a warning saying that the cropping parameters do not exist yet. Therefore, a cropping tool can be used to crop the data while also saving the crop ranges into a class attribute for later scans.

In [ ]:
sps.crop_tool()

Press crop applies the cropping to the test image, and stores the cropping information in the class.
Load the scan again to apply it to all images:

In [ ]:
res_xarray = sps.load_scan(
    scan=4450, # Scan number for an example mirror scan
    path = path,
    crop=True
)

The data may be visualised as a 2D plot by slicing the xarray in 2 dimensions keeping the third fixed. In the plot below, the Angle vs Ekin data is plotted for a fixed mirrorX value.

In [ ]:
sps.result.coords

In [ ]:
plt.figure()
sps.result[:,:,0].plot()
plt.show()

The metadata associated with the scan is added as an attribute to the xarray

In [ ]:
sps.result.attrs["metadata"].keys()

3D scans, where the images are recorded as a function of a third parameter (generally delay or in this case, mirrorX), can also be loaded with an option to average only the given iterations passed as a list or slice object. 

In [ ]:
sps.load_scan(
    scan=4450,
    path=path,
    iterations=[0]
)

In [ ]:
plt.figure()
sps.result[:,:,0].plot()
plt.show()

Another useful functionality is to load a 3D scan as a function of iterations averaged over the scan parameter (in this case, mirrorX). This is done using the check_scan method

In [ ]:
res_xarray_check = sps.check_scan(
    scan=4450,
    delays=1, # for a fixed delay of index, 1
    path=path
)

In [ ]:
res_xarray_check

## Conversion to NeXus
This required the [nexusutils](https://github.com/nomad-coe/nomad-parser-nexus) package to be installed in the active kernel. Once installed, the convert function can be imported and called using the mpes reader which makes use of the NXmpes contributed application definition for NeXus for the Multidimensional Photoemission Spectroscopy community.

In [ ]:
from pynxtools.dataconverter.convert import convert

In [ ]:
convert(input_file=["../tests/data/phoibos_config.json", # config file for translating local metadata paths to NeXus paths
                "../tests/data/phoibos_eln_data.yaml" # ELN file that adds/overwrites additional metadata
        ],
        objects=sps.result, # xarray object obtained from the specsscan loader
        reader='mpes',
        nxdl='NXmpes',
        output='spectest.mpes.nxs')

## View the data with H5Web
H5Web is a tool for visualizing any data in the h5 data format. Since the NeXus format builds opon h5 it can be used to view this data as well. We just import the package and call H5Web with the output filename from the convert command above. 

You can also view this data with the H5Viewer or other tools from your local filesystem.

In [ ]:
from jupyterlab_h5web import H5Web

In [ ]:
H5Web('spectest.mpes.nxs')